In [ ]:
import numpy as np
import pandas as pd
import gc

train = pd.read_csv("train.csv",skipinitialspace=True,index_col='ncodpers')
train.info(verbose=True)

In [ ]:
train_bool = train.iloc[:,:408].astype(bool)
train_bool.to_csv("train_bool.csv",index=True)

In [ ]:
import numpy as np
import pandas as pd
import gc

train = pd.read_csv("train_bool.csv",skipinitialspace=True,index_col='ncodpers')
train.info(verbose=True)


In [ ]:
products=set([i[:-11] for i in train.columns])

for col in products:
    cols=[i for i in train.columns if col in i]
    train[col]=train[cols].any(axis=1)
    train=train.drop(columns=cols)

train.info(verbose=True)

In [ ]:
train

In [ ]:
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
freq_items=fpgrowth(train, min_support=0.00001, use_colnames=True, max_len=None, verbose=0)


In [ ]:
freq_items

In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(freq_items, metric="confidence", min_threshold=0.2)


In [ ]:
rules.sort_values('confidence')

In [ ]:
rules["consequents_len"] = rules["consequents"].apply(lambda x: len(x))

In [ ]:
rules_conseq1 = rules[rules["consequents_len"]==1]
#del rules
#gc.collect()
rules_conseq1.sort_values('confidence',ascending=False)

In [ ]:
items = [j for i in rules_conseq1['antecedents'] for j in i]

In [ ]:
rules_conseq1['consequents'].unique()

In [ ]:
import time
start_time = time.time()

out=pd.DataFrame(columns = ['ncodpers','p1','p2','p3','p4','p5','p6','p7'])

for index,row in train.head(100).iterrows():
    setD=set([i for i,x in enumerate(row) if x])
    indMatch=[i for i,x in enumerate(rules_conseq1['antecedents']) if (setD&x)==x]
    if len(indMatch)<7:
        indMatch=[i for i,x in enumerate(rules_conseq1['antecedents']) if len(setD&x)>0]
    conseq=rules_conseq1.iloc[indMatch,[1,5]]
    selectedConseq=[train.columns[list(i)[0]] for i in conseq.groupby(['consequents'])['confidence'].max().nlargest(7).index]
    if len(selectedConseq)<7:
        continue
    insRow= pd.Series([index]+selectedConseq, index = out.columns)
    out=out.append(insRow,ignore_index = True)


print("--- %s seconds ---" % (time.time() - start_time))

out
#df.groupby(['school_name', 'user_id'])['like_id'].nunique().sum(level=0).nlargest(5)



In [ ]:
import time
start_time = time.time()
from multiprocessing import Pool
from contextlib import closing

_func = None

def worker_init(func):
  global _func
  _func = func
  

def worker(x):
  return _func(x)


def xmap(func, iterable, processes=None):
   with closing( Pool(processes, initializer=worker_init, initargs=(func,)) ) as p:
    return p.map(worker, iterable)


out=pd.DataFrame(columns = ['ncodpers','p1','p2','p3','p4','p5','p6','p7'])

def findTop7(ind,row):
    insRow=pd.Series([ind]+[None]*7,index=out.columns)
    setD=set([i for i,x in enumerate(row) if x])
    indMatch=[i for i,x in enumerate(rules['antecedents']) if (setD&x)==x]
    if len(indMatch)<7:
        indMatch=[i for i,x in enumerate(rules['antecedents']) if len(setD&x)>0]
    conseq=rules.iloc[indMatch,[1,5]]
    conseq=pd.DataFrame({'consequents':np.repeat(conseq.consequents.values, len(conseq.confidence)),
                        'confidence':np.concatenate(conseq.confidence.values)})
    print(conseq)
    #selectedConseq=[train.columns[list(i)[0]] for i in conseq.groupby(['consequents'])['confidence'].max().nlargest(7).index]
    #insRow= pd.Series([ind]+selectedConseq, index = out.columns)
    return insRow

out=out.append(xmap(lambda x: findTop7(x[0],x[1]),train.head(20).iterrows()))

print("--- %s seconds ---" % (time.time() - start_time))
out
#out.to_csv("arm_out.csv",index=False)


In [ ]:
#out['ncodpers']=out['ncodpers'].astype(int)
out
out.to_csv("arm_out.csv",index=False)

In [ ]:
out = pd.read_csv("arm_out.csv")
test = pd.read_csv("test.csv",index_col='ncodpers_')
test.info(verbose=True)

In [ ]:
out

In [ ]:
out_join=test.merge(out,how='left', left_index=True, right_on='ncodpers')
out_join.info(verbose=True)

In [ ]:
out_join=out_join.iloc[:,-8:]

In [ ]:
out_join=out_join.set_index('ncodpers')

In [ ]:
out_join

In [ ]:
outfile = out_join.apply(lambda x:  '' if x.isnull().values.any() else ' '.join(x),axis=1)
outfile

In [ ]:
outfile.name = 'added_products'
outfile.index.rename('ncodpers',inplace=True)
outfile.to_csv('outfile_arm1.csv',sep=',',index=True)